#  RT2 Jupyter notebook User Interface 

This Code is developed from the RT1 assignment. 

The project depicts a robot behaviour where it can switch to different modalities of functionalities. One of the modality is to move the robot to a user defined position (x,y coordinates)autonomously. Another modality is to drive the robot by manually controlling through the keyboard keys. From these modalities, plotting the robot position, laser scanner data, reached and non-reached positions.  

In [ ]:
from ipywidgets import Layout, Button, Box, VBox, HBox
import ipywidgets as widgets
import rospy
import actionlib
from std_msgs.msg import Twist
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
import jupyros as jr
from std_msgs.msg import String
from IPython.display import display
from geometry_msgs.msg import Twist

import matplotlib.pyplot as plt
import numpy as np
import tf
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan

from move_base_msgs.msg import MoveBaseActionGoal

pub3 = rospy.Publisher("/cmd_vel", Twist, queue_size = 1000)
pub1 = rospy.Publisher('/move_base/goal', MoveBaseActionGoal, queue_size = 1000)

vel_linear = 1.0
vel_angular = 0.0
x = 0.0
y = 0.0
rate = 0.001
flag7 = 0
flag5 = 0
flag8 = 0

rospy.set_param('/flag5', flag5)
robot_speed = Twist()


In [ ]:
class DataVisualization:
    def __init__(self):
        self.fig, self.ax = plt.subplots()

        self.pl, = plt.plot([], [], 'go')
        self.pp, = plt.plot([], [], 'ro')
        self.x_pos, self.y_pos = [] , []
        self.x_scan, self.y_scan = [] , []

        def pp_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.pp
    
    def pl_init(self): 
        self.ax.set_xlim(-10, 730)
        self.ax.set_ylim(0, 15)
        return self.pl
    
    def update_odom(self, frame):
        self.pp.set_data(self.x_pos, self.y_pos)
        return self.pp
    

    def update_scan(self, frame):
        self.pl.set_data(self.x_scan, self.y_scan)
        return self.pl
    

    def odom_callback(self, msg):
        if flag7 == 0:
            self.x_pos.clear();
            self.y_pos.clear();
        self.y_pos.append(msg.pose.pose.position.y)
        self.x_pos.append(msg.pose.pose.position.x)
    

    def scan_callback(self, msg):
        self.x_scan.clear();
        self.y_scan.clear();
        for i in range(len(msg.ranges)):
            self.y_scan.append(msg.ranges[i])
            self.x_scan.append(len(msg.ranges)-i)

In [ ]:
def clear_out(arg):
    output.clear_output()
    send_out.clear_output()
    canc_out.clear_output()

In [ ]:
def enable_assistance(arg):
        
    global flag8
    flag8 = 1
    rospy.set_param('/flag4', 0)
    rospy.set_param('/flag1', 1)
    clear_output()
    ui()

In [ ]:
def disable_assistance(arg):
    global flag8
    flag8 = 0
    rospy.set_param('/flag4', 0)
    rospy.set_param('/flag1', 0)
    clear_output() 
    ui()

In [ ]:
def enable_assistance_2(arg):
    global flag8
    flag8 = 1
    rospy.set_param('/flag4', 0)
    rospy.set_param('/flag1', 1)
    clear_output() 
    take_the_wheel(_)

In [ ]:
def disable_assistance_2(arg):
    global flag8
    flag8 = 0
    rospy.set_param('/flag4', 0)
    rospy.set_param('/flag1', 0)
    clear_output() 
    take_the_wheel(_)

In [ ]:
def forward(arg):
    global vel_linear
    rospy.set_param('/flag4', 0)
    rospy.set_param('/flag6', 0)
    
    robot_speed.linear.x = vel_linear.value
    robot_speed.angular.z = 0
    pub3.publish(robot_speed)
    
    with output:
        output.clear_output()
        if vel_linear.value == 0.0:
            print(" \n The robot is still \n ")
        else: 
            print(" \n The robot is moving forward \n ")

In [ ]:
def backward(arg):
    global vel_linear
    rospy.set_param('/flag4', 0)
    rospy.set_param('/flag6', 3)
    
    robot_speed.linear.x = -vel_linear.value
    robot_speed.angular.z = 0
    pub3.publish(robot_speed)
    
    with output:
        output.clear_output()
        if vel_linear.value == 0.0:
            print(" Linear velocity is 0, so the robot is still ")
        else: 
            print(" The robot is moving backwards")

In [ ]:
def curve_left(arg):
    global vel_linear
    global vel_angular
    rospy.set_param('/flag4', 0)
    rospy.set_param('/flag6', 1)
    
    robot_speed.linear.x = vel_linear.value
    robot_speed.angular.z = vel_angular.value
    pub3.publish(robot_speed)
    
    with output:
        output.clear_output()
        if vel_linear.value == 0.0:
            print(" The linear velocity is 0, the robot is moving ")
        elif vel_angular.value == 0.0:
            print(" The angular velocity is 0, the robot is still moving ")
        elif vel_angular.value == 0.0 and vel_linear.value == 0:
            print(" the robot is not moving with 0 linear and angular velocity")
        else: 
            print(" Curving left")

In [ ]:
def curve_right(arg):
    global vel_linear
    global vel_angular
    rospy.set_param('/flag4', 0)
    rospy.set_param('/flag6', 2)
    
    robot_speed.linear.x = vel_linear.value
    robot_speed.angular.z = -vel_angular.value
    pub3.publish(robot_speed)
    
    with output:
        output.clear_output()
        if vel_linear.value == 0.0:
            print("The linear velocity is 0, the robot is moving")
        elif vel_angular.value == 0.0:
            print("The angular velocity is 0, the robot is still moving")
        elif vel_angular.value == 0.0 and vel_linear.value == 0:
            print("the robot is not moving with 0 linear and angular velocity")
        else: 
            print(" curving right ")

In [ ]:
def turn_left(arg):
    global vel_angular
    rospy.set_param('/flag4', 0)
    rospy.set_param('/flag6', 3)
    
    robot_speed.linear.x = 0.0
    robot_speed.angular.z = vel_angular.value
    pub3.publish(robot_speed)
    
    with output:
        output.clear_output()
        if vel_angular.value == 0.0:
            print(" robot not moving with angular velocity set to 0 ")
        else: 
            print(" turning left ")

In [ ]:
def turn_right(arg):
    global vel_angular
    rospy.set_param('/flag4', 0)
    rospy.set_param('/flag6', 3)
    
    robot_speed.linear.x = 0.0
    robot_speed.angular.z = -vel_angular.value
    pub3.publish(robot_speed)
    
    with output:
        output.clear_output()
        if vel_angular.value == 0.0:
            print("robot not moving with angular velocity set to 0 ")
        else: 
            print(" turning right ")

In [ ]:
def stop(arg):
    rospy.set_param('/flag4', 0)
    rospy.set_param('/flag6', 3)
    
    robot_speed.linear.x = 0.0
    robot_speed.angular.z = 0.0
    pub3.publish(robot_speed)
    
    with output:
        output.clear_output()

In [ ]:
def quit(arg):
    rospy.set_param('/flag4', 0)
    rospy.set_param('/flag6', 3)
    
    robot_speed.linear.x = 0.0
    robot_speed.angular.z = 0.0
    pub3.publish(robot_speed)
        
    clear_output() 
    ui()

#Manual Driving modality 

In [ ]:
def take_the_wheel(arg):
    clear_output() 
    print("\n Manual driving \n")
    
    global flag8 
    global vel_linear
    global vel_angular
    
    rospy.set_param('/flag4', 0)
    flag5 = rospy.get_param("/flag5")
    if flag5 == 1:
        rospy.set_param('/flag5', 0)
        flag5 = 0
    if isinstance(vel_linear, widgets.widgets.widget_float.FloatSlider):
        vel_linear = vel_linear.value
    if isinstance(vel_angular, widgets.widgets.widget_float.FloatSlider):
        vel_angular = vel_angular.value
    vel_linear = widgets.FloatSlider( 
        value = vel_linear,
        min = 0.0,
        max = 2.0,
        step = 0.1,
        description = 'Linear velocity',
        disabled = False,
        continuous_update = False,
        orientation = 'horizontal',
        readout = True,
        readout_format='.1f',
    ) 
    vel_angular = widgets.FloatSlider( 
        value = vel_angular,
        min = 0.0,
        max = 5.0,
        step = 0.1,
        description = 'Angular velocity',
        disabled = False,
        continuous_update = False,
        orientation = 'horizontal',
        readout = True,
        readout_format='.1f',
    )
    forward_b = widgets.Button(description = 'Go on', layout = button_layout, style = button_style)
    forward_b.style.button_color = 'green'
    
    backward_b = widgets.Button(description = 'Go back', layout = button_layout, style = button_style)
    backward_b.style.button_color = 'green'
    
    curve_left_b = widgets.Button(description = 'Curve left', layout = button_layout, style = button_style)
    curve_left_b.style.button_color = 'green'
    
    curve_right_b = widgets.Button(description = 'Curve right', layout = button_layout, style = button_style)
    curve_right_b.style.button_color = 'green'
    
    turn_left_b = widgets.Button(description = 'Turn left', layout = button_layout, style = button_style)
    turn_left_b.style.button_color = 'green'
    
    turn_right_b = widgets.Button(description = 'Turn right', layout = button_layout, style = button_style)
    turn_right_b.style.button_color = 'green'
    
    stop_b = widgets.Button(description = 'Stop', layout = button_layout, style = button_style)
    stop_b.style.button_color = 'yellow'
    
    with_assistance_b = widgets.Button(description = 'Enable driving assistance', layout = button_layout, style = button_style)
    with_assistance_b.style.button_color = 'green'
    
    without_assistance_b = widgets.Button(description = 'Disable driving assistance', layout = button_layout, style = button_style)
    without_assistance_b.style.button_color = 'orange'
    
    quit_b = widgets.Button(description = 'Quit', layout = button_layout, style = button_style)
    quit_b.style.button_color = 'red'
    
    forward_b.on_click(forward)
    backward_b.on_click(backward)
    curve_left_b.on_click(curve_left)
    curve_right_b.on_click(curve_right)
    turn_left_b.on_click(turn_left)
    turn_right_b.on_click(turn_right)
    stop_b.on_click(stop)
    with_assistance_b.on_click(enable_driving_assistance_2)
    without_assistance_b.on_click(disable_driving_assistance_2)
    quit_b.on_click(back)
    
    if flag8 == 0:
        display(vel_linear, vel_angular, 
            widgets.HBox((curve_left_b, forward_b, curve_right_b)), 
            widgets.HBox((turn_left_b, backward_b, turn_right_b,)),
            widgets.HBox((stop_b, with_assistance_b, quit_b)), 
            output)
    elif flag8 == 1:
        display(vel_linear, vel_angular, 
            widgets.HBox((curve_left_b, forward_b, curve_right_b)), 
            widgets.HBox((turn_left_b, backward_b, turn_right_b,)),
            widgets.HBox((stop_b, without_assistance_b, quit_b)), 
            output)

In [ ]:
def cancel_goal(arg):
    goal_flag = rospy.get_param("/goal_flag")
    if goal_flag == 1:
        rospy.set_param('/goal_flag', 0)
        goal_flag = 0
        
        with canc_out:    
            canc_out.clear_output()
            print("goal cancelled ")
    else: 
        with canc_out:    
            canc_out.clear_output()
            print(" No goal has been set ")

In [ ]:
def send_coordinates(arg):
    goal_pos = MoveBaseActionGoal()    
    goal_pos.goal.target_pose.header.frame_id = "map"
    goal_pos.goal.target_pose.pose.orientation.w = 1;
    goal_pos.goal.target_pose.pose.position.x = x.value
    goal_pos.goal.target_pose.pose.position.y = y.value
    
    pub_goal.publish(goal_pos)
    rospy.set_param('/goal_flag', 1)
    
    with send_out:    
        send_out.clear_output()
        print(" goal set ")

In [ ]:
def set_coordinates(arg):   
    global x
    global y
    clear_output()  
    
    send_b = widgets.Button(description = 'Send', layout = button_layout, style = button_style) 
    send_b.style.button_color = 'green'
    back_b = widgets.Button(description = 'Back', layout = button_layout, style = button_style)
    back_b.style.button_color = 'red'
 
    
    x = widgets.BoundedFloatText(
    value = 0.0,
    min = -100,
    step = 0.1,
    description = 'x :',
    disabled = False
    )
    y = widgets.BoundedFloatText(
    value = 0.0,
    min = -100,
    step = 0.1,
    description = 'y :',
    disabled = False
    )
    
    back_b.on_click(back)
    send_b.on_click(send_coordinates)
    
    display(x, y, send_b, back_b, send_out)

In [ ]:
def plot_target(arg):
    reached_goal = rospy.get_param("/reached_goal")
    non_reached_goal = rospy.get_param("/non_reached_goal")
    targets = ["Reached", "Non-reached"]
    number = [reached_goal, non_reached_goal]
    
    fig = plt.figure()
    fig.canvas.manager.set_window_title('Data plotting')
    plt.bar(targets, number, color ='blue', width = 0.5)
    plt.title("Targets info")
    plt.xlabel("Targets")
    plt.ylabel("Number")
    plt.show(block = False)
    
    while plt.fignum_exists(1):
        plt.pause(rate) 
        new_reached_goal = rospy.get_param("/reached_goal")
        new_non_reached_goal = rospy.get_param("/non_reached_goal")
        if (new_reached_goal != reached_goal) or (new_non_reached_goal != non_reached_goal):
            reached_goal = new_reached_goal
            non_reached_goal = new_non_reached_goal
            number = [reached_goal, non_reached_goal]
            plt.bar(targets, number, color ='blue', width = 0.5)

In [ ]:
def plot_laserscan(arg):
    vis = Visualiser() 
    ani = FuncAnimation(vis.fig, 
                        vis.update_scan, 
                        init_func = vis.pl_init)
    plt.title("Laser scanner")
    plt.ylabel("Distance")
    sub = rospy.Subscriber('/scan', LaserScan, vis.scan_callback)
    plt.show(block = False)
    
    while plt.fignum_exists(1):
        plt.pause(rate)

In [ ]:
def plot_position(arg):
    vis = Visualiser() 
    ani = FuncAnimation(vis.fig, 
                        vis.update_odom, 
                        init_func = vis.pp_init)
    plt.title("Robot position")
    plt.xlabel("x")
    plt.ylabel("y")
    sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
    plt.show(block = False)
    
    while plt.fignum_exists(1):
        plt.pause(rate)

In [ ]:
def data_visualization(arg):  
    global flag7
    clear_output() 
    pos_b = widgets.Button(description = 'Plot position data', layout = button_layout, style = button_style)
    pos_b.style.button_color = 'green'
    pos_b.on_click(plot_position) 
    
    scan_b = widgets.Button(description = 'Plot laser scanner data', layout = button_layout, style = button_style)
    scan_b.style.button_color = 'green'
    scan_b.on_click(plot_laserscan)
    
    target_b = widgets.Button(description = 'Plot targets info', layout = button_layout, style = button_style)
    target_b.style.button_color = 'green'
    target_b.on_click(plot_target) 
    
    
    back_b = widgets.Button(description = 'Back', layout = button_layout, style = button_style)
    back_b.style.button_color = 'red'
    back_b.on_click(back)  
    
    if flag7 == 0:
        display(pos_b, scan_b, target_b, pos_scan_b, with_hist_b, back_b)    
    elif flag7 == 1:
        display(pos_b, scan_b, target_b, pos_scan_b, without_hist_b, back_b)

In [ ]:
def ui():
    global flag8
    global flag9 
    
    print("\n This is a USER INTERFACE Modality!\n")
    exit_b = widgets.Button(description = 'Exit', layout = button_layout, style = button_style)
    exit_b.style.button_color = 'red'
    exit_b.on_click(terminate) 

    plot_b = widgets.Button(description = 'Plot data', layout = button_layout, style = button_style)
    plot_b.style.button_color = 'green'
    plot_b.on_click(data_visualization) 
    

    goal_b = widgets.Button(description = 'Insert new coordinates to reach', layout = button_layout, style = button_style)
    goal_b.style.button_color = 'green'
    goal_b.on_click(set_coordinates) 
    
    canc_b = widgets.Button(description = 'Cancel the current goal', layout = button_layout, style = button_style)
    canc_b.style.button_color = 'green'
    canc_b.on_click(cancel_goal)

    drive_b = widgets.Button(description = 'Manual driving', layout = button_layout, style = button_style)
    drive_b.style.button_color = 'green'
    drive_b.on_click(take_the_wheel)
    

    with_assistance_b = widgets.Button(description = 'Enable driving assistance', layout = button_layout, style = button_style)
    with_assistance_b.style.button_color = 'green'
    with_assistance_b.on_click(enable_assistance) 

    without_assistance_b = widgets.Button(description = 'Disable driving assistance', layout = button_layout, style = button_style)
    without_assistance_b.style.button_color = 'orange'
    without_assistance_b.on_click(disable_assistance) 

    clear_b = widgets.Button(description = 'Clear button outputs', layout = button_layout, style = button_style)
    clear_b.style.button_color = 'yellow'
    clear_b.on_click(clear_out) 

    if flag9 == 0:
        if flag8 == 0:
            display(exit_b, plot_b, goal_b, canc_b, canc_out, drive_b, with_assistance_b, clear_b)   
        elif flag8 == 1:
            display(exit_b, plot_b, goal_b, canc_b, canc_out, drive_b, without_assistance_b, clear_b)
    elif flag9 == 1:
        if flag8 == 0:
            display(exit_b, goal_b, canc_b, canc_out, drive_b, with_assistance_b, clear_b)   
        elif flag8 == 1:
            display(exit_b, goal_b, canc_b, canc_out, drive_b, without_assistance_b, clear_b)


In [ ]:
def main():
    rospy.init_node("rt2_jupyter_interface")    
    ui()
    
if __name__ == '__main__':
    main()

In [ ]:
def update_target(arg):
    reached_goal = rospy.get_param("/reached_goal")
    non_reached_goal = rospy.get_param("/non_reached_goal")
    targets = ["Reached", "Non-reached"]
    number = [reached_goal, non_reached_goal]
    
    plt.bar(targets, number, color ='blue', width = 0.5)

In [ ]:
if flag9 == 1:
    
    vis1 = Visualiser() 
    ani1 = FuncAnimation(vis1.fig, 
                        vis1.update_odom, 
                        init_func = vis1.pp_init)
    plt.title("Robot position")
    plt.xlabel("x")
    plt.ylabel("y")
    sub = rospy.Subscriber('/odom', Odometry, vis1.odom_callback)
    plt.show(block = False)

In [ ]:
import os
from jupyros import ros3d

view = ros3d.Viewer() # ROS 3D object
rc = ros3d.ROSConnection(url = "ws://localhost:9090") # Websocket (9090:rosbridge)
tf_client = ros3d.TFClient(ros = rc, fixed_frame = 'map') # Client connection 

# Visualize the map, the laser and the path to follow
map_view = ros3d.OccupancyGrid(topic = "/map", ros = rc, tf_client = tf_client) 
laser_view = ros3d.LaserScan(topic = "/scan", ros = rc, tf_client = tf_client) 
path = ros3d.Path(topic = "/move_base/NavfnROS/plan", ros = rc, tf_client = tf_client)

urdf = ros3d.URDFModel(ros = rc, tf_client = tf_client,
    path = os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000'))

g = ros3d.GridModel() # Grid object

# Show the model
view.objects = [g, laser_view, map_view, path, urdf]
view